##### Copyright 2023 The TF-Agents Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Checkpointer y PolicySaver

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/10_checkpointer_policysaver_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/10_checkpointer_policysaver_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/10_checkpointer_policysaver_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/10_checkpointer_policysaver_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Introducción

`tf_agents.utils.common.Checkpointer` es una utilidad que sirve para guardar o cargar el estado del entrenamiento, el estado de la política y el estado de replay_buffer en un almacenamiento local o desde él.

`tf_agents.policies.policy_saver.PolicySaver` es una herramienta que se usa para guardar o cargar solo la política y es más liviano que `Checkpointer`. También puede usar `PolicySaver` para implementar el modelo sin necesidad de conocer el código que creó la política.

En este tutorial, usaremos DQN para entrenar un modelo, luego usaremos `Checkpointer` y `PolicySaver` para mostrar cómo podemos almacenar y cargar los estados y el modelo de forma interactiva. tenga en cuenta que usaremos la nueva herramienta y formato saved_model de TF2.0 para `PolicySaver`.


## Preparación

Si todavía no ha instalado las siguientes dependencias, ejecute estos comandos:

In [ ]:
#@test {"skip": true}
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg python-opengl
!pip install pyglet
!pip install 'imageio==2.4.0'
!pip install 'xvfbwrapper==0.2.9'
!pip install tf-agents[reverb]

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import imageio
import io
import matplotlib
import matplotlib.pyplot as plt
import os
import shutil
import tempfile
import tensorflow as tf
import zipfile
import IPython

try:
  from google.colab import files
except ImportError:
  files = None
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

tempdir = os.getenv("TEST_TMPDIR", tempfile.gettempdir())

In [ ]:
#@test {"skip": true}
# Set up a virtual display for rendering OpenAI gym environments.
import xvfbwrapper
xvfbwrapper.Xvfb(1400, 900, 24).start()

## Agente DQN

Ahora vamos a configurar el agente DQN, igual que en el colab anterior. Los detalles se ocultan de forma predeterminada, ya que no son parte central de este colab, pero puede hacer clic en "SHOW CODE" (Mostrar código) para ver los detalles.

### Hiperparámetros

In [ ]:
env_name = "CartPole-v1"

collect_steps_per_iteration = 100
replay_buffer_capacity = 100000

fc_layer_params = (100,)

batch_size = 64
learning_rate = 1e-3
log_interval = 5

num_eval_episodes = 10
eval_interval = 1000

### Entorno

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

### Agente

In [ ]:
#@title
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

global_step = tf.compat.v1.train.get_or_create_global_step()

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=global_step)
agent.initialize()

### Recopilación de datos

In [ ]:
#@title
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

collect_driver = dynamic_step_driver.DynamicStepDriver(
    train_env,
    agent.collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=collect_steps_per_iteration)

# Initial data collection
collect_driver.run()

# Dataset generates trajectories with shape [BxTx...] where
# T = n_step_update + 1.
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

iterator = iter(dataset)

### Cómo entrenar el agente

In [ ]:
#@title
# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

def train_one_iteration():

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_driver.run()

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience)

  iteration = agent.train_step_counter.numpy()
  print ('iteration: {0} loss: {1}'.format(iteration, train_loss.loss))

### Generación de videos

In [ ]:
#@title
def embed_gif(gif_buffer):
  """Embeds a gif file in the notebook."""
  tag = '<img src="data:image/gif;base64,{0}"/>'.format(base64.b64encode(gif_buffer).decode())
  return IPython.display.HTML(tag)

def run_episodes_and_create_video(policy, eval_tf_env, eval_py_env):
  num_episodes = 3
  frames = []
  for _ in range(num_episodes):
    time_step = eval_tf_env.reset()
    frames.append(eval_py_env.render())
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = eval_tf_env.step(action_step.action)
      frames.append(eval_py_env.render())
  gif_file = io.BytesIO()
  imageio.mimsave(gif_file, frames, format='gif', fps=60)
  IPython.display.display(embed_gif(gif_file.getvalue()))

### Cómo generar un video

Para comprobar el funcionamiento de la política, genere un video.

In [ ]:
print ('global_step:')
print (global_step)
run_episodes_and_create_video(agent.policy, eval_env, eval_py_env)

## Configuración de Checkpointer y PolicySaver

Ahora estamos listos para usar Checkpointer y PolicySaver.

### Checkpointer


In [ ]:
checkpoint_dir = os.path.join(tempdir, 'checkpoint')
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=global_step
)

### PolicySaver

In [ ]:
policy_dir = os.path.join(tempdir, 'policy')
tf_policy_saver = policy_saver.PolicySaver(agent.policy)

### Cómo entrenar una iteración

In [ ]:
#@test {"skip": true}
print('Training one iteration....')
train_one_iteration()

### Cómo guardar en un punto de verificación

In [ ]:
train_checkpointer.save(global_step)

### Cómo restaurar un punto de verificación

Para que esto funcione, todo el conjunto de objetos debe recrearse de la misma forma que cuando se creó el punto de verificación.

In [ ]:
train_checkpointer.initialize_or_restore()
global_step = tf.compat.v1.train.get_global_step()

Además, guarde la política y expórtela a una ubicación

In [ ]:
tf_policy_saver.save(policy_dir)

Puede cargar la política incluso si no sabe qué agente o red se usaron para crearla. Esto facilita mucho la implementación de la política.

Cargue la política guardada y compruebe su funcionamiento.

In [ ]:
saved_policy = tf.saved_model.load(policy_dir)
run_episodes_and_create_video(saved_policy, eval_env, eval_py_env)

## Exportación e importación

El resto del colab nos ayudará a exportar/importar los directorios de checkpointer y de políticas para que podamos seguir entrenando más adelante e implementar el modelo sin tener que repetir el entrenamiento.

Ahora puede volver a 'Cómo entrenar una iteración' y entrenar unas cuantas veces más para poder entender la diferencia más adelante. Una vez que empiece a ver resultados ligeramente mejores, continúe adelante.

In [ ]:
#@title Create zip file and upload zip file (double-click to see the code)
def create_zip_file(dirname, base_filename):
  return shutil.make_archive(base_filename, 'zip', dirname)

def upload_and_unzip_file_to(dirname):
  if files is None:
    return
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    shutil.rmtree(dirname)
    zip_files = zipfile.ZipFile(io.BytesIO(uploaded[fn]), 'r')
    zip_files.extractall(dirname)
    zip_files.close()

Cree un archivo comprimido desde el directorio de puntos de verificación.

In [ ]:
train_checkpointer.save(global_step)
checkpoint_zip_filename = create_zip_file(checkpoint_dir, os.path.join(tempdir, 'exported_cp'))

Descargue el archivo zip.

In [ ]:
#@test {"skip": true}
if files is not None:
  files.download(checkpoint_zip_filename) # try again if this fails: https://github.com/googlecolab/colabtools/issues/469

Después de entrenar un par de veces (10-15 veces), descargue el archivo zip del punto de verificación, y diríjase a "Tiempo de ejecución &gt; Reiniciar y ejecutar todo" para reiniciar el entrenamiento, y vuelva a esta celda. Ahora ya puede cargar el archivo zip descargado y continuar el entrenamiento.

In [ ]:
#@test {"skip": true}
upload_and_unzip_file_to(checkpoint_dir)
train_checkpointer.initialize_or_restore()
global_step = tf.compat.v1.train.get_global_step()

Una vez que haya cargado el directorio de puntos de verificación, vuelva a "Cómo entrenar una iteración" para continuar el entrenamiento o vuelva a "Cómo generar un vídeo" para comprobar el rendimiento de la política cargada.

Otra opción es guardar la política (modelo) y restaurarla. A diferencia de checkpointer, no podrá continuar con el entrenamiento, pero podrá implementar el modelo. Tenga en cuenta que el archivo descargado es mucho más pequeño que el de checkpointer.

In [ ]:
tf_policy_saver.save(policy_dir)
policy_zip_filename = create_zip_file(policy_dir, os.path.join(tempdir, 'exported_policy'))

In [ ]:
#@test {"skip": true}
if files is not None:
  files.download(policy_zip_filename) # try again if this fails: https://github.com/googlecolab/colabtools/issues/469

Cargue el directorio de políticas descargado (exported_policy.zip) y compruebe cómo funciona la política guardada.

In [ ]:
#@test {"skip": true}
upload_and_unzip_file_to(policy_dir)
saved_policy = tf.saved_model.load(policy_dir)
run_episodes_and_create_video(saved_policy, eval_env, eval_py_env)


## SavedModelPyTFEagerPolicy

Si no desea usar la política de TF, entonces también puede usar saved_model directamente con el entorno de Python mediante el uso de `py_tf_eager_policy.SavedModelPyTFEagerPolicy`.

Tenga en cuenta que esto solo funciona cuando el modo eager está habilitado.

In [ ]:
eager_py_policy = py_tf_eager_policy.SavedModelPyTFEagerPolicy(
    policy_dir, eval_py_env.time_step_spec(), eval_py_env.action_spec())

# Note that we're passing eval_py_env not eval_env.
run_episodes_and_create_video(eager_py_policy, eval_py_env, eval_py_env)

## Cómo convertir políticas en TFLite

Consulte [Convertidor de TensorFlow Lite](https://www.tensorflow.org/lite/convert) para obtener más información.

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(policy_dir, signature_keys=["action"])
tflite_policy = converter.convert()
with open(os.path.join(tempdir, 'policy.tflite'), 'wb') as f:
  f.write(tflite_policy)

### Cómo ejecutar una inferencia sobre el modelo TFLite

Consulte [Inferencia de TensorFlow Lite](https://tensorflow.org/lite/guide/inference) para obtener más información.

In [ ]:
import numpy as np
interpreter = tf.lite.Interpreter(os.path.join(tempdir, 'policy.tflite'))

policy_runner = interpreter.get_signature_runner()
print(policy_runner._inputs)

In [ ]:
policy_runner(**{
    '0/discount':tf.constant(0.0),
    '0/observation':tf.zeros([1,4]),
    '0/reward':tf.constant(0.0),
    '0/step_type':tf.constant(0)})